In [1]:
from glob import glob
import os
import cv2
import csv


# read files

In [2]:
#os.listdir("./dataset/20200902_day_shadow_01_Class 1. 정상주행/frame/0_20200902_day_shadow_01")
imgs = glob("./dataset/20200902_day_shadow_01_Class 1. 정상주행/frame/0_20200902_day_shadow_01/*.jpg")
csv_file = glob("./dataset/20200902_day_shadow_01_Class 1. 정상주행/frame/0_20200902_day_shadow_01/*.csv")

In [19]:

f = open(csv_file[0], 'r', encoding="utf-8")
rdr = csv.reader(f)
filename = []
features = []
for line in rdr:
    #print(line)
    #print("len(line): ", len(line))
    try:
        filename.append(line[0])
        features.append(line[1:])
    except:
        pass
f.close()

In [10]:
int_features = []


for item in features:
    temp = []
    for i in range(0, len(features[0])-1,2):
        #print(i, i+1)

        pair = (int(item[i]),  int(item[i+1]))
        temp.append(pair)
    
    int_features.append(temp)
#     print(temp)
#     print(len(temp))

# get positive features

In [11]:
features_ps = []

In [12]:
for item in int_features:
    temp  = []
    for tup in item:
        if tup[0] == -1:
            pass
        else:
            temp.append(tup)
            
    features_ps.append(temp)

In [13]:
features_x = []
features_y = []
for item in features_ps:
    #item => list of tuples
#     print(item)
    temp_x = [] 
    temp_y = []
    for tup in item:
#         print("tup[0]: ", tup[0])
#         print("tup[1]:", tup[1])
        temp_x.append(tup[0])
        temp_y.append(tup[1])
    features_x.append(temp_x)
    features_y.append(temp_y)

# print(features_x)
# print(features_y)

# get min, max of features

In [14]:
features_minmax = []
for i in range(0, len(features_ps)):
    assert len(features_x[i]) == len(features_y[i])
    
#     print("xmin: ", min(features_x[i]))
#     print("ymin: ", min(features_y[i]))
#     print("xmax: ", max(features_x[i]))
#     print("ymax: ", max(features_y[i]))
#     print("---")
    # [xmin, ymin, xmax, ymax]
    features_minmax.append([min(features_x[i]), min(features_y[i]), max(features_x[i]), max(features_y[i])])

# landmark, bbox 그리기

In [15]:
import numpy as np
temp  = np.array(features_x[0])

In [16]:
temp.mean()

542.6666666666666

In [17]:
temp.std()

30.014811158641145

In [18]:
for item in imgs:
    img = cv2.imread(item, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w , c = img.shape
    index = filename.index(os.path.basename(item))
    
    x_mean = int(np.array(features_x[index]).mean())
    y_mean = int(np.array(features_y[index]).mean())
    x_std = int(np.array(features_x[index]).std())
    y_std = int(np.array(features_y[index]).std())
    
    xmin, ymin, xmax, ymax = features_minmax[index]
    
    xmin = xmin - x_std
    ymin = ymin - y_std
    xmax = xmax + x_std
    ymax = ymax + y_std

    if xmin <= 0:
        xmin = 0
    if ymin <= 0:
        ymin = 0        
    if xmax >= w:
        xmax = w
    if ymax > h:
        ymax = h
        
    # draw landmark
    for tup in int_features[index]:
        if tup[0] == -1:
            pass
        cv2.circle(img, tup, 4, (0,0,255), -1)
    
    # draw bbox
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0,255,0), 3 )
    save_path = "./dataset/20200902_day_shadow_01_Class 1. 정상주행/frame/0_20200902_day_shadow_01/output/"+ os.path.basename(item) +".jpg"
    cv2.imwrite(save_path, img)